<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" role="tab" aria-controls="home">Table of Content</h3>

* [1. Reading the Data](#1)
* [2. Exploratory Data Analysis](#2)
    - [2.1 Correlation matrix](#2.1)       
    - [2.2 Scatter plot](#2.2)
    - [2.3 Missing values](#2.3) 
    - [2.4 Outliers](#2.4)     

This notebook aims at predicting sales prices and practice feature engineering, RFs, and gradient boosting in the framework of the Kaggle competition [House Prices - Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview).

___
**_Description and context:_**
Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence. 
With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.


In [1]:
# Loading libraries 

%matplotlib notebook

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
from warnings import filterwarnings
filterwarnings('ignore')

# Loading variables

train_filename = '../data/train.csv'
test_filename = '../data/test.csv'


<a id="1"></a>
<font color="darkslateblue" size=+2.5><b>1. Reading the Data</b></font> 

<a href="#top" class="btn btn-primary" role="button">Table of Content</a>

In [2]:
train_data, test_data = pd.read_csv(train_filename), pd.read_csv(test_filename)
display(train_data.head())
train_data.info()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

One can find the description of the data (79) in [data_description.txt](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data):

* **_MSSubClass_**: Identifies the type of dwelling involved in the sale.	
* **_MSZoning_**: Identifies the general zoning classification of the sale.
* **_LotFrontage_**: Linear feet of street connected to property
* **_LotArea_**: Lot size in square feet
* **_Street_**: Type of road access to property
* **_Alley_**: Type of alley access to property
* **_LotShape_**: General shape of property
* **_LandContour_**: Flatness of the property
* **_Utilities_**: Type of utilities available
* **_LotConfig_**: Lot configuration
* **_LandSlope_**: Slope of property
* **_Neighborhood_**: Physical locations within Ames city limits
* **_Condition1_**: Proximity to various conditions
* **_Condition2_**: Proximity to various conditions (if more than one is present)
* **_BldgType_**: Type of dwelling
* **_HouseStyle_**: Style of dwelling
* **_OverallQual_**: Rates the overall material and finish of the house
* **_OverallCond_**: Rates the overall condition of the house
* **_YearBuilt:_** Original construction date
* **_YearRemodAdd_**: Remodel date (same as construction date if no remodeling or additions)
* **_RoofStyle_**: Type of roof
* **_RoofMatl_**: Roof material
* **_Exterior1st_**: Exterior covering on house
* **_Exterior2nd_**: Exterior covering on house (if more than one material)
* **_MasVnrType_**: Masonry veneer type
* **_MasVnrArea_**: Masonry veneer area in square feet
* **_ExterQual_**: Evaluates the quality of the material on the exterior 
* **_ExterCond_**: Evaluates the present condition of the material on the exterior
* **_Foundation_**: Type of foundation
* **_BsmtQual_**: Evaluates the height of the basement
* **_BsmtCond_**: Evaluates the general condition of the basement
* **_BsmtExposure_**: Refers to walkout or garden level walls
* **_BsmtFinType1_**: Rating of basement finished area
* **_BsmtFinSF1_**: Type 1 finished square feet
* **_BsmtFinType2_**: Rating of basement finished area (if multiple types)
* **_BsmtFinSF2_**: Type 2 finished square feet
* **_BsmtUnfSF_**: Unfinished square feet of basement area
* **_TotalBsmtSF_**: Total square feet of basement area
* **_Heating_**: Type of heating
* **_HeatingQC_**: Heating quality and condition
* **_CentralAir_**: Central air conditioning		
* **_Electrical_**: Electrical system
* **_1stFlrSF_**: First Floor square feet 
* **_2ndFlrSF_**: Second floor square feet
* **_LowQualFinSF_**: Low quality finished square feet (all floors)
* **_GrLivArea_**: Above grade (ground) living area square feet
* **_BsmtFullBath_**: Basement full bathrooms
* **_BsmtHalfBath_**: Basement half bathrooms
* **_FullBath_**: Full bathrooms above grade
* **_HalfBath_**: Half baths above grade
* **_Bedroom_**: Bedrooms above grade (does NOT include basement bedrooms)
* **_Kitchen_**: Kitchens above grade
* **_KitchenQual_**: Kitchen quality      	
* **_TotRmsAbvGrd_**: Total rooms above grade (does not include bathrooms)
* **_Functional_**: Home functionality (Assume typical unless deductions are warranted)
* **_Fireplaces_**: Number of fireplaces
* **_FireplaceQu_**: Fireplace quality
* **_GarageType_**: Garage location
* **_GarageYrBlt_**: Year garage was built		
* **_GarageFinish_**: Interior finish of the garage
* **_GarageCars_**: Size of garage in car capacity
* **_GarageArea_**: Size of garage in square feet
* **_GarageQual_**: Garage quality
* **_GarageCond_**: Garage condition
* **_PavedDrive_**: Paved driveway
* **_WoodDeckSF_**: Wood deck area in square feet
* **_OpenPorchSF_**: Open porch area in square feet
* **_EnclosedPorch_**: Enclosed porch area in square feet
* **_3SsnPorch_**: Three season porch area in square feet
* **_ScreenPorch_**: Screen porch area in square feet
* **_PoolArea_**: Pool area in square feet
* **_PoolQC_**: Pool quality
* **_Fence_**: Fence quality
* **_MiscFeature_**: Miscellaneous feature not covered in other categories
* **_MiscVal_**: $Value of miscellaneous feature
* **_MoSold_**: Month Sold (MM)
* **_YrSold_**: Year Sold (YYYY)
* **_SaleType_**: Type of sale
* **_SaleCondition_**: Condition of sale

<a id="2"></a>
<font color="darkslateblue" size=+2.5><b>2. Exploratory Data Analysis</b></font> 

<a href="#top" class="btn btn-primary" role="button">Table of Content</a>

In [3]:
train_data.drop(columns='Id',inplace=True);  test_data.drop(columns='Id',inplace=True) # drop ID column

<a id="2.1"></a>
<font color="dimgrey" size=+2.0><b>2.1 Correlation matrix</b></font>

<a href="#top" class="btn btn-primary" role="button">Table of Content</a>

In [31]:
#plt.figure(figsize=(30,10))

corrmat = train_data.corr()

heatmap = sns.heatmap(corrmat, annot = True, cmap = "YlGnBu", fmt = '.1g', 
                      linewidths = 1, linecolor='black', square = False,
                      cbar = False, cbar_kws = {'orientation': 'horizontal'}) # cmap="YlGnBu",plt.cm.Reds
fig = heatmap.get_figure()


fig.savefig("correlation_matrix.png")

In [32]:
#plt.figure(figsize=(30,10))

# Display a subset of the most correlated features with Salesprice

corr_limit = 1./3.

corrmat = train_data.corr(method='spearman')
target = abs(corrmat["SalePrice"])
relevant_features = target[target>corr_limit]  # let's keep only highly correlated features

train_data_zoom = train_data[relevant_features.index]
corrmat = train_data_zoom.corr(method='spearman')

heatmap = sns.heatmap(corrmat, annot = True, cmap = "YlGnBu", fmt = '.2g', 
                      linewidths = 1, linecolor='black', square = False,
                      cbar = False, cbar_kws = {'orientation': 'horizontal'}) # cmap="YlGnBu",plt.cm.Reds
fig = heatmap.get_figure()

fig.savefig("correlation_matrix_zoom.png")

In [6]:
# I got rid of the features which are strongly correlated but less than with Salesprice with another feature

# I have chosen a limit of 2./3. (arbitrary) -> To automatize

features = train_data_zoom.drop(['YearRemodAdd','GarageYrBlt','1stFlrSF','TotRmsAbvGrd'],1)



<a id="2.2"></a>
<font color="dimgrey" size=+2.0><b>2.2 Scatter plot</b></font>

<a href="#top" class="btn btn-primary" role="button">Table of Content</a>

In [35]:
#plt.figure(figsize=(30,10))

sns.set()
sns.pairplot(features, size = 1.0)

#plt.show()

<a id="2.3"></a>
<font color="dimgrey" size=+2.0><b>2.3 Missing values</b></font>

<a href="#top" class="btn btn-primary" role="button">Table of Content</a>

In [8]:
# Missing values

nan_col = train_data.isnull().sum().sort_values(ascending=False)
nan_col_per = (train_data.isnull().sum()/train_data.isnull().count()*100).sort_values(ascending=False)
nan_col_table = pd.concat([nan_col,np.round(nan_col_per,1)], axis=1, keys=['Missing Values', '% of Total Values'])

display(nan_col_table.head(20))

,Missing Values,% of Total Values
PoolQC,1453,99.5
MiscFeature,1406,96.3
Alley,1369,93.8
Fence,1179,80.8
FireplaceQu,690,47.3
LotFrontage,259,17.7
GarageType,81,5.5
GarageCond,81,5.5
GarageFinish,81,5.5
GarageQual,81,5.5


Drop the variables where more than 15% of the values are missing. Here we do not assume that the client needs 
to keep absolutely one of the variables (it should depend upon the importance of the variable).

In [9]:
# Filter the columns with more than X% missing values
limit = 15
nan_col_abovelimit = nan_col_table[nan_col_table['% of Total Values'] >= limit]
nan_col_belowlimit = nan_col_table[nan_col_table['% of Total Values'] < limit]
train_data_ = train_data.drop(columns = nan_col_abovelimit.index)

Drop rows where there are missing values

In [10]:
train_data_clean = train_data_.dropna(subset = nan_col_belowlimit.index, axis = 0)
train_data_clean.reset_index(inplace=True,drop=True)
display(train_data_clean)

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,60,RL,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,...,0,0,0,0,0,8,2007,WD,Normal,175000
1334,20,RL,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,...,0,0,0,0,0,2,2010,WD,Normal,210000
1335,70,RL,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1336,20,RL,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,112,0,0,0,0,4,2010,WD,Normal,142125


<a id="2.4"></a>
<font color="dimgrey" size=+2.0><b>2.4 Outliers</b></font>

<a href="#top" class="btn btn-primary" role="button">Table of Content</a>

In [17]:
train_data_numerical = train_data_clean.select_dtypes(include = [np.number])

# Outliers removed (except from the target, Saleprice)
outliers = train_data_numerical.drop(['SalePrice'],1)
z_scores = stats.zscore(outliers)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
outliers = outliers[filtered_entries]
train_data_numerical = outliers.merge(train_data_numerical['SalePrice'], left_index=True, right_index=True)
train_data_numerical.reset_index(inplace=True,drop=True) 
display(train_data_numerical)

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,8450,7,5,2003,2003,196.0,706,0,150,...,0,61,0,0,0,0,0,2,2008,208500
1,60,11250,7,5,2001,2002,162.0,486,0,434,...,0,42,0,0,0,0,0,9,2008,223500
2,60,14260,8,5,2000,2000,350.0,655,0,490,...,192,84,0,0,0,0,0,12,2008,250000
3,20,10084,8,5,2004,2005,186.0,1369,0,317,...,255,57,0,0,0,0,0,8,2007,307000
4,20,11200,5,5,1965,1965,0.0,906,0,134,...,0,0,0,0,0,0,0,2,2008,129500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,50,11767,4,7,1910,2000,0.0,0,0,560,...,168,24,0,0,0,0,0,5,2007,112000
956,20,9262,8,5,2008,2009,194.0,0,0,1573,...,0,36,0,0,0,0,0,5,2009,287090
957,20,7500,7,5,2004,2005,0.0,410,0,811,...,0,113,0,0,0,0,0,10,2009,185000
958,60,7917,6,5,1999,2000,0.0,0,0,953,...,0,40,0,0,0,0,0,8,2007,175000


In [34]:
#plt.figure(figsize=(30,10))

corrmat = train_data_numerical.corr()

heatmap = sns.heatmap(corrmat, annot = True, cmap = "YlGnBu", fmt = '.1g', 
                      linewidths = 1, linecolor='black', square = False,
                      cbar = False, cbar_kws = {'orientation': 'horizontal'}) # cmap="YlGnBu",plt.cm.Reds
fig = heatmap.get_figure()